## License plate detection

### Importing Libraries

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from os.path import splitext,basename
from keras.models import model_from_json
import glob


from keras.preprocessing.image import ImageDataGenerator
from keras.applications import MobileNetV2
from keras.layers import AveragePooling2D
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split

import matplotlib.gridspec as gridspec
from sklearn.preprocessing import LabelEncoder

from local_utils import detect_lp

### Function to load character detection model

In [ ]:
def load_model(path):
    try:
        path = splitext(path)[0]
        with open('%s.json' % path, 'r') as json_file:
            model_json = json_file.read()
        model = model_from_json(model_json, custom_objects={})
        model.load_weights('%s.h5' % path)
        print("Model loaded successfully!")
        return model
    except Exception as e:
        print(e)  
wpod_net_path = "./wpod-net.json"
wpod_net = load_model(wpod_net_path)

### Load Images For Testing

In [ ]:
image_paths = glob.glob("Kenyan_Vehicles_License_Plates/*.png")

def preprocess_image(image_path,resize=False):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img / 255
    if resize:
        img = cv2.resize(img, (224,224))
    return img


fig = plt.figure(figsize=(12, 8))
cols = 3
rows = 3
fig_list = []

for i, image_path in enumerate(image_paths):
    if i >= cols * rows:
        break

    fig_list.append(fig.add_subplot(rows, cols, i + 1))
    title = splitext(basename(image_path))[0]
    fig_list[-1].set_title(title)
    img = preprocess_image(image_path)
    plt.axis(False)
    plt.imshow(img)

plt.tight_layout()
plt.show()

### License plate detection

In [ ]:
def get_plate(image_path, Dmax=500, Dmin=400):
    vehicle = preprocess_image(image_path)
    ratio = float(max(vehicle.shape[:2])) / min(vehicle.shape[:2])
    side = int(ratio * Dmin)
    bound_dim = min(side, Dmax)
    _ , LpImg, _, cor = detect_lp(wpod_net, vehicle, bound_dim, lp_threshold=0.5)
    return LpImg, cor

# Retrieve license plate image and its coordinates
test_image = image_paths[20]
LpImg,cor = get_plate(test_image)
print("Detect %i plate in"%len(LpImg),splitext(basename(test_image))[0])
print("Coordinate of plate in image: \n", cor)

# Visualize the image and the extracted plate
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.axis(False)
plt.imshow(preprocess_image(test_image))
plt.subplot(1,2,2)
plt.axis(False)
plt.imshow(LpImg[0])

plt.savefig("Image and License Plate Detection_2.jpg",dpi=300)

### Image Augmentation

In [ ]:
if (len(LpImg)): #check if there is an license image
    # Scale, calculates absolute values, and convert to 8-bit
    plate_image = cv2.convertScaleAbs(LpImg[0], alpha=(255.0))

    # grayscale and blur the image
    gray = cv2.cvtColor(plate_image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray,(7,7),0)

    # inversed binary thresholding and dilating
    binary = cv2.threshold(blur, 180, 255,
                         cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    kernel3 = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    thre_mor = cv2.morphologyEx(binary, cv2.MORPH_DILATE, kernel3)


# visualize preprocessed license plate
fig = plt.figure(figsize=(12,7))
plt.rcParams.update({"font.size":18})
grid = gridspec.GridSpec(ncols=2,nrows=3,figure = fig)
plot_image = [plate_image, gray, blur, binary,thre_mor]
plot_name = ["plate_image","gray","blur","binary","dilation"]

for i in range(len(plot_image)):
    fig.add_subplot(grid[i])
    plt.axis(False)
    plt.title(plot_name[i])
    if i ==0:
        plt.imshow(plot_image[i])
    else:
        plt.imshow(plot_image[i],cmap="gray")

plt.savefig("License plate preprocessing_2.png", dpi=300)

### Character Detection

In [ ]:
def sort_contours(cnts,reverse = False):
    i = 0
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
                                        key=lambda b: b[1][i], reverse=reverse))
    return cnts

cont, _  = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

test_roi = plate_image.copy()

# a list to append segmented character images
crop_characters = []

# standard filter for width and height of character
digit_w, digit_h = 30, 60

for c in sort_contours(cont):
    (x, y, w, h) = cv2.boundingRect(c)
    ratio = h/w
    if 1<=ratio<=3.5: # Only select contour with defined ratio
        if h/plate_image.shape[0]>=0.5: # Select contours height larger than 50% of the plate
            # Draw bounding box around character
            cv2.rectangle(test_roi, (x, y), (x + w, y + h), (0, 255,0), 2)

            # Seperate characters and append to list
            curr_num = thre_mor[y:y+h,x:x+w]
            curr_num = cv2.resize(curr_num, dsize=(digit_w, digit_h))
            # convert characters to binary
            _, curr_num = cv2.threshold(curr_num, 220, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            crop_characters.append(curr_num)

print("Detected {} characters.".format(len(crop_characters)))
fig = plt.figure(figsize=(10,6))
plt.axis(False)
plt.imshow(test_roi)
plt.savefig('Bounded and Segmented Characters_2.png',dpi=300)

### Display the segmented characters in the list one by one, in a grid

In [ ]:
fig = plt.figure(figsize=(14,4))
grid = gridspec.GridSpec(ncols=len(crop_characters),nrows=1,figure=fig)

for i in range(len(crop_characters)):
    fig.add_subplot(grid[i])
    plt.axis(False)
    plt.imshow(crop_characters[i],cmap="gray")
plt.savefig("Segmented Characters_2.png",dpi=300)

### Character Dataset For Training

In [ ]:
import os

dataset_paths = glob.glob("dataset_characters/CNN_letter_Dataset/**/*.jpg")

cols = 4
rows = 3
fig = plt.figure(figsize=(10, 8))
plt.rcParams.update({"font.size": 14})
grid = gridspec.GridSpec(ncols=cols, nrows=rows, figure=fig)

# Calculate the number of images in the dataset
num_images = len(dataset_paths)

# Check if there are enough images to fill the grid
if num_images >= (cols * rows):
    # Generate random indices
    np.random.seed(45)
    rand = np.random.choice(num_images, size=(cols * rows), replace=False)

    # Plot characters and their labels
    for i in range(cols * rows):
        fig.add_subplot(grid[i])
        image = load_img(dataset_paths[rand[i]])
        label = dataset_paths[rand[i]].split(os.path.sep)[-2]
        plt.title('"{:s}"'.format(label))
        plt.axis(False)
        plt.imshow(image)

    plt.savefig("Sample of Dataset Characters.png", dpi=300)
else:
    print("Not enough images in the dataset to fill the grid.")

### Image Preprocessing for Modelling

In [ ]:
X=[]# character data in array form
labels=[]# labels

for image_path in dataset_paths:
    label = image_path.split(os.path.sep)[-2]
    image=load_img(image_path,target_size=(80,80))
    image=img_to_array(image)

    X.append(image)
    labels.append(label)

X = np.array(X,dtype="float16")
labels = np.array(labels)

print("Found {:d} images with {:d} classes".format(len(X),len(set(labels))))


# encode the labels for equal representation
lb = LabelEncoder()
lb.fit(labels)
labels = lb.transform(labels)
y = to_categorical(labels)

# save label file for reuse
np.save('license_character_classes.npy', lb.classes_)

# split 10% of data as validation set
(trainX, testX, trainY, testY) = train_test_split(X, y, test_size=0.10, stratify=y, random_state=42)

# data augumentation to diversify the training set
image_gen = ImageDataGenerator(rotation_range=10,
                              width_shift_range=0.1,
                              height_shift_range=0.1,
                              shear_range=0.1,
                              zoom_range=0.1,
                              fill_mode="nearest"
                              )

### Creating a Model

In [ ]:
def create_model(learning_rate=1e-4, decay=1e-4/25, training=False, output_shape=y.shape[1]):
    baseModel = MobileNetV2(weights="imagenet",
                            include_top=False,
                            input_tensor=Input(shape=(80, 80, 3)))

    headModel = baseModel.output
    headModel = AveragePooling2D(pool_size=(3, 3))(headModel)
    headModel = Flatten(name="flatten")(headModel)
    headModel = Dense(128, activation="relu")(headModel)
    headModel = Dense(128, activation="relu")(headModel)
    headModel = Dense(128, activation="relu")(headModel)
    headModel = Dropout(0.5)(headModel)
    headModel = Dense(output_shape, activation="softmax")(headModel)

    model = Model(inputs=baseModel.input, outputs=headModel)

    if training:
        # Define trainable layers in the base model
        for layer in baseModel.layers:
            layer.trainable = True

        # Compile the model with learning rate and loss function
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

    return model

# Initialize hyperparameters
INIT_LR = 1e-4
EPOCHS = 5

In [ ]:
model = create_model(learning_rate=INIT_LR, decay=INIT_LR/EPOCHS, training=True)

### Training the model

In [ ]:
EPOCHS = 20
BATCH_SIZE = 300

my_checkpointer = [
                EarlyStopping(monitor='val_loss', patience=5, verbose=0),
                ModelCheckpoint(filepath="License_character_recognition_weight (1).h5", verbose=1, save_weights_only=True)
                ]

result = model.fit(image_gen.flow(trainX, trainY, batch_size=BATCH_SIZE),
                   steps_per_epoch=len(trainX) // BATCH_SIZE,
                   validation_data=(testX, testY),
                   validation_steps=len(testX) // BATCH_SIZE,
                   epochs=EPOCHS, callbacks=my_checkpointer)

### Model Validation

In [ ]:
fig = plt.figure(figsize=(14,5))
grid=gridspec.GridSpec(ncols=2,nrows=1,figure=fig)
fig.add_subplot(grid[0])
plt.plot(result.history['accuracy'], label='training accuracy')
plt.plot(result.history['val_accuracy'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()

fig.add_subplot(grid[1])
plt.plot(result.history['loss'], label='training loss')
plt.plot(result.history['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()

plt.savefig("Training and Validation Curves.jpg",dpi=300)

### Prediction

In [ ]:
model_json = model.to_json()
with open("MobileNets_character_recognition_model.json", "w") as json_file:
  json_file.write(model_json)
json_file = open('MobileNets_character_recognition_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights("License_character_recognition_weight (1).h5")
print("Model loaded successfully!")

labels = LabelEncoder()
labels.classes_ = np.load('license_character_classes.npy')
print("Labels loaded successfully!")

def predict_from_model(image,model,labels):
    image = cv2.resize(image,(80,80))
    image = np.stack((image,)*3, axis=-1)
    prediction = labels.inverse_transform([np.argmax(model.predict(image[np.newaxis,:]))])
    return prediction
# display the each of segmented characters in a grid and the predicted labels as the title.
fig = plt.figure(figsize=(15,3))
cols = len(crop_characters)
grid = gridspec.GridSpec(ncols=cols,nrows=1,figure=fig)

final_string = ''
for i,character in enumerate(crop_characters):
    fig.add_subplot(grid[i])
    title = np.array2string(predict_from_model(character,model,labels))
    plt.title('{}'.format(title.strip("'[]"),fontsize=10))
    final_string+=title.strip("'[]")
    plt.axis(False)
    plt.imshow(character,cmap='gray')

print(final_string)
plt.savefig('Recognized License Plate_2.png', dpi=300)